In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import pandas_datareader.data as web
import datetime
import pandas_datareader.data as web
import datetime
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc



start = datetime.datetime(2020,1,1)
end = datetime.datetime(2020,12,3)
df = web.DataReader(['AMZN','GOOGL','FB','PFE','MRNA','BNTX'],
                    'stooq', start=start, end=end)
# df=df.melt(ignore_index=False, value_name="price").reset_index()
df = df.stack().reset_index()

In [2]:
df.head()

Attributes,level_0,Symbols,Close,High,Low,Open,Volume
0,2020-01-02,AMZN,94.9005,94.9005,93.2075,93.7500,8.071820e+07
1,2020-01-02,GOOGL,68.4340,68.4340,67.3245,67.4205,2.728530e+07
2,2020-01-02,PFE,35.0488,35.2381,34.8126,35.1943,1.749704e+07
3,2020-01-02,MRNA,19.2300,19.8100,18.8800,19.5700,1.229434e+06
4,2020-01-02,BNTX,38.1343,38.1343,35.0241,35.4104,1.410105e+05


In [3]:
df.columns

Index(['level_0', 'Symbols', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Attributes')

In [4]:
df.level_0.min(), df.level_0.max()

(Timestamp('2020-01-02 00:00:00'), Timestamp('2020-12-03 00:00:00'))

In [5]:
df.dtypes

Attributes
level_0    datetime64[ns]
Symbols            object
Close             float64
High              float64
Low               float64
Open              float64
Volume            float64
dtype: object

In [6]:
df["month"] = df["level_0"].apply(lambda x: x.month)

In [7]:
df.rename(columns={"level_0":"date"},inplace=True)

In [8]:
df.Symbols.unique()

array(['AMZN', 'GOOGL', 'PFE', 'MRNA', 'BNTX'], dtype=object)

In [11]:
df.head()

Attributes,date,Symbols,Close,High,Low,Open,Volume,month
0,2020-01-02,AMZN,94.9005,94.9005,93.2075,93.7500,8.071820e+07,1
1,2020-01-02,GOOGL,68.4340,68.4340,67.3245,67.4205,2.728530e+07,1
2,2020-01-02,PFE,35.0488,35.2381,34.8126,35.1943,1.749704e+07,1
3,2020-01-02,MRNA,19.2300,19.8100,18.8800,19.5700,1.229434e+06,1
4,2020-01-02,BNTX,38.1343,38.1343,35.0241,35.4104,1.410105e+05,1


In [13]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP]
                  #MOBİLE LAYOUT,
                  #meta_tags=[{'name': 'viewport',
                  #          'content': 'width=device-width, initial-scale=1.0'}]
                 )
app.layout = dbc.Container([
    dbc.Row([  #1.row row kullandıktqan sonra kolon kullan, birden fazla column kullanılabilir ama burada direkt title olduğu için bütün rowu kapsar 
        dbc.Col(html.H1("Stock Market 2020 Performance Dashboard",
                        className='text-center text-primary mb-4'),
               width=12)
    ]),
    
    dbc.Row([

        dbc.Col([
            dcc.Dropdown(id='my-dp', multi=False, value='AMZN',
                         options=[{'label':x, 'value':x}
                                  for x in sorted(df['Symbols'].unique())],
                         ),
            dcc.Graph(id='line', figure={})
        ], width={'size':5, 'offset':1, 'order':1},
           #xs=12, sm=12, md=12, lg=5, xl=5
        ),

        dbc.Col([
            dcc.Dropdown(id='my-dp2', multi=True, value=['PFE','BNTX'],
                         options=[{'label':x, 'value':x}
                                  for x in sorted(df['Symbols'].unique())],
                         ),
            dcc.Graph(id='line2', figure={})
        ], width={'size':5, 'offset':0, 'order':2},
           #xs=12, sm=12, md=12, lg=5, xl=5
        ),

    ], justify='start'),
    
    
    dbc.Row([
        dbc.Col([
            html.P("Select Company Stock:",
                   style={"textDecoration": "underline"}),
            dcc.Checklist(id='my-checklist', value=['FB', 'GOOGL', 'AMZN'],
                          options=[{'label':x, 'value':x}
                                   for x in sorted(df['Symbols'].unique())],
                          #inline=True,
                          labelClassName="mr-3"),
            dcc.Graph(id='my-hist', figure={}),
        ], width={'size':8, 'offset':1},
           #xs=12, sm=12, md=12, lg=5, xl=5
        )
    ], align="center")
], fluid=True #default fluid false ancak bu rowlarda space bırakır sağda ve solda başlangıç bitiş boşluk koyar bunun için width kullanılabilir offset ile 
)


@app.callback(
    Output('line','figure'),
    Input('my-dp','value')
)
def update_graph(stock_slctd):
    dff = df[df["Symbols"]==stock_slctd]
    figln = px.line(data_frame=dff,x='date',y='High')
    return figln   #output line-fig id için figure alanına koyar


@app.callback(
    Output('line2','figure'),
    Input('my-dp2','value')
)

def update_graph(stock_slctd):
    dff = df[df['Symbols'].isin(stock_slctd)]
    figln2 = px.line(dff, x='date', y='Open', color='Symbols')
    return figln2


# Histogram
@app.callback(
    Output('my-hist', 'figure'),
    Input('my-checklist', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols'].isin(stock_slctd)]
    #dff = dff[dff['date']=='2020-12-03']
    fighist = px.histogram(dff, x='Symbols', y='Close',
                          animation_frame="month",
                          range_y=[0,4000])
    return fighist









if __name__ == "__main__":
    app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

